In [7]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(context='talk', style='ticks',
        color_codes=True, rc={'legend.frameon': False})

%matplotlib inline

In [8]:
data = pd.read_parquet('/ccs/home/pstjohn/project_work/swissprot/parsed_swissprot.parquet')

In [9]:
data.head()

,accession,EMBL,RefSeq,KEGG,InterPro,Pfam,NCBI Taxonomy,length,sequence,subcellularLocalization
0,Q6GZX4,AY548484,YP_031579.1,vg:2947773,IPR007031,PF04947,654924,256,MAFSAEDVLKEYDRRRRMEALLLSLYYPNDRKLLDYKEWSPPRVQV...,
1,Q6GZX3,AY548484,YP_031580.1,vg:2947774,IPR004251,PF03003,654924,320,MSIIGATRLQNDKSDTYSAGPCYAGGCSAFTPRGTCGKDWDLGEQT...,Host membrane
2,Q197F8,DQ643392,YP_654574.1,vg:4156251,None,None,345201,458,MASNTVSAQGGSNRPVRDFSNIQDVAQFLLFDPIWNEQPGSIVPWK...,
3,Q197F7,DQ643392,YP_654575.1,vg:4156252,None,None,345201,156,MYQAINPCPQSWYGSPQLEREIVCKMSGAPHYPNYYPVHPNALGGA...,
4,Q6GZX2,AY548484,YP_031581.1,vg:2947775,None,None,654924,438,MARPLLGKTSSVRRRLESLSACSIFFFLRKFCQKMASLVFLNSPVY...,


In [10]:
subcell = data[data['subcellularLocalization'] != ''].loc[:, ['accession', 'subcellularLocalization']]

In [11]:
lens = [len(item.split('\n')) for item in subcell['subcellularLocalization']]

In [21]:
expanded_subcell = pd.DataFrame(
    {"accession" : np.repeat(subcell['accession'].values, lens), 
     "subcellularLocalization" : np.hstack(subcell['subcellularLocalization'].str.split('\n'))})
expanded_subcell = expanded_subcell.drop_duplicates()

In [23]:
expanded_subcell.subcellularLocalization.value_counts().head(50)

Cytoplasm                            160222
Nucleus                               38424
Cell membrane                         33947
Secreted                              30632
Cell inner membrane                   26877
Plastid                               15599
Membrane                              14912
Endoplasmic reticulum membrane         6612
Mitochondrion inner membrane           6453
Mitochondrion                          5616
Virion                                 4482
Host nucleus                           3914
Cell projection                        3907
Chromosome                             3738
Host cytoplasm                         3510
Cell junction                          3227
Periplasm                              3099
Golgi apparatus membrane               3064
Golgi apparatus                        2733
Cytoplasmic vesicle                    2640
Virion membrane                        2271
Cell outer membrane                    2264
Cellular thylakoid membrane     

In [24]:
one_hot = pd.get_dummies(expanded_subcell.subcellularLocalization)
one_hot['accession'] = expanded_subcell['accession']
grouped_one_hot = one_hot.groupby('accession').sum()

In [28]:
(grouped_one_hot.max() == 1).all()

True

In [34]:
subset = grouped_one_hot.loc[:, grouped_one_hot.sum(0) > 3000]

In [35]:
subset.shape

(344004, 18)

In [36]:
test = subset.sample(10000, random_state=1)
valid = subset[~subset.index.isin(test.index)].sample(10000, random_state=1)
train = subset[~subset.index.isin(test.index) & ~subset.index.isin(valid.index)]

In [37]:
train.to_csv('/ccs/home/pstjohn/project_work/swissprot/subcellular/train.csv.gz', compression='gzip')
valid.to_csv('/ccs/home/pstjohn/project_work/swissprot/subcellular/valid.csv.gz', compression='gzip')
test.to_csv('/ccs/home/pstjohn/project_work/swissprot/subcellular/test.csv.gz', compression='gzip')